# Notebook de Traduction Anglais-Français (Seq2Seq) avec Embeddings

Ce notebook implémente un modèle encodeur-décodeur pour la traduction de l'anglais vers le français. 

**Objectif :** Remplacer l'approche "caractère par caractère" et "one-hot" par :
1.  Un **vocabulaire de mots** (limité à 2056 tokens).
2.  Des couches `Embedding` pour représenter les mots.

## 1. Téléchargement et Chargement des Données

Nous allons d'abord télécharger le fichier `fra.txt` qui contient les paires de phrases anglais-français.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_file
import numpy as np
import re
import string

# Télécharger le fichier de données
data_path = get_file(
    "fra.txt",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/fra-eng.zip",
    extract=True
)
# Le chemin est maintenant un répertoire, nous devons trouver le fichier .txt à l'intérieur
data_path = data_path.replace(".zip", "/fra.txt")

# Lire le fichier et afficher les premières lignes
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

print("Exemples de lignes du fichier fra.txt:")
for line in lines[:5]:
    print(line)

print(f"\nNombre total de paires de phrases: {len(lines)}")

## 2. Création du Dataset (Paires Anglais-Français)

Nous allons parser le fichier et créer une liste de paires `(input_text, target_text)`. 

Pour que le décodeur sache quand commencer et finir, nous ajoutons des tokens spéciaux :
* `[start]` au début des séquences cibles (français).
* `[end]` à la fin des séquences cibles.

Nous allons également limiter la taille du dataset pour un entraînement plus rapide.

In [ ]:
num_samples = 30000  # On utilise 30 000 échantillons pour un entraînement rapide
input_texts = []
target_texts = []

for line in lines[: min(num_samples, len(lines) - 1)]:
    try:
        input_text, target_text, _ = line.split("\t")
        # Ajout des tokens de début et de fin pour la cible (français)
        target_text = "[start] " + target_text + " [end]"
        input_texts.append(input_text)
        target_texts.append(target_text)
    except ValueError:
        # Ignorer les lignes mal formées
        pass

print("\nExemple de paire (Entrée, Cible) après traitement:")
print(f"Entrée (Anglais): {input_texts[10]}")
print(f"Cible (Français): {target_texts[10]}")
print(f"\nTaille du dataset utilisé: {len(input_texts)}")

## 3, 4 & 5. Tokenization et Vectorisation (Remplacement des étapes 3, 4, 5)

Au lieu de trouver des *caractères* et de faire du *one-hot* (étapes 3, 4, 5 de votre plan), nous allons utiliser la couche `TextVectorization` de Keras. C'est la méthode moderne pour :
1.  Créer un vocabulaire de mots (étape 3).
2.  Transformer les chaînes de caractères en chaînes d'entiers (étape 4).
3.  Préparer les données pour une couche `Embedding` (remplace l'étape 5).

Nous fixons la taille du vocabulaire à **2056** comme demandé.

In [ ]:
VOCAB_SIZE = 2056
EMBEDDING_DIM = 256
LATENT_DIM = 256  # États internes de l'LSTM, comme demandé (étape 6a)
BATCH_SIZE = 64

# Fonction de standardisation pour nettoyer le texte
def custom_standardization(input_string):
    # Mettre en minuscule
    lowercase = tf.strings.lower(input_string)
    # Retirer les balises HTML (au cas où)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    # Gérer la ponctuation. [start] et [end] sont des mots, ne pas les séparer.
    # Garder les crochets pour nos tokens spéciaux
    punctuation_to_remove = '"#$%&()*+,-./:;<=>@\\^_`{|}~'
    # Retirer la ponctuation
    no_punctuation = tf.strings.regex_replace(
        stripped_html, f'[{re.escape(punctuation_to_remove)}]', ''
    )
    # Séparer les points d'interrogation et d'exclamation
    no_punctuation = tf.strings.regex_replace(no_punctuation, "([?.!,])", r" \1 ")
    return no_punctuation

# Création de la couche de vectorisation pour l'Anglais (Entrée)
input_vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    standardize=custom_standardization
)

# Création de la couche de vectorisation pour le Français (Cible)
target_vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    standardize=custom_standardization
)

# Adapter (construire le vocabulaire) les couches sur nos textes
input_vectorization.adapt(input_texts)
target_vectorization.adapt(target_texts)

print("\n--- Vocabulaire Français (Cible) ---")
print(target_vectorization.get_vocabulary()[:20])

print("\n--- Exemple de vectorisation ---")
sample_input = input_texts[10]
sample_target = target_texts[10]
print(f"Texte Anglais: {sample_input}")
print(f"Vectorisé: {input_vectorization([sample_input]).numpy()}")
print(f"Texte Français: {sample_target}")
print(f"Vectorisé: {target_vectorization([sample_target]).numpy()}")

### Création du `tf.data.Dataset`

Nous transformons nos listes en un `tf.data.Dataset` pour l'efficacité. C'est ici que nous préparons les paires pour le *teacher forcing* :

* `decoder_input`: `[start] Je suis...`
* `decoder_target`: `Je suis... [end]`

Le modèle apprendra à prédire le mot `Je` en voyant `[start]`, puis `suis` en voyant `Je`, etc.

In [ ]:
def vectorize_pair(input_text, target_text):
    input_vec = input_vectorization([input_text])
    target_vec = target_vectorization([target_text])
    return input_vec[0], target_vec[0] # [0] pour retirer la dimension batch

def format_dataset(input_vec, target_vec):
    # Prépare les paires pour le teacher forcing
    # input_vec: [1, 2, 3]
    # target_vec: [10, 11, 12, 13] ([start], mot1, mot2, [end])
    
    # decoder_input: [10, 11, 12] ([start], mot1, mot2)
    decoder_input = target_vec[:-1]
    
    # decoder_target: [11, 12, 13] (mot1, mot2, [end])
    decoder_target = target_vec[1:]
    
    # Retourne: ({input_1: ..., input_2: ...}, output)
    return ({'encoder_input': input_vec, 'decoder_input': decoder_input}, decoder_target)

# Créer le dataset de base
dataset = tf.data.Dataset.from_tensor_slices((input_texts, target_texts))

# Vectoriser les textes
# Note: tf.py_function est nécessaire car TextVectorization n'est pas (encore) parfaitement 
# compatible avec .map() si les formes ne sont pas statiques.
# Une alternative est de tout vectoriser en mémoire d'abord, mais c'est moins flexible.

# Pour simplifier et garantir la compatibilité, nous allons vectoriser en mémoire (style NumPy)
input_vecs = input_vectorization(input_texts)
target_vecs = target_vectorization(target_texts)

# Trouver la longueur maximale pour le padding
max_input_seq_len = input_vecs.shape[1]
max_target_seq_len = target_vecs.shape[1]

print(f"\nMax seq length (input): {max_input_seq_len}")
print(f"Max seq length (target): {max_target_seq_len}")

# Créer le dataset final à partir des tenseurs vectorisés
dataset = tf.data.Dataset.from_tensor_slices((input_vecs, target_vecs))
dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=2048).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Vérifier un batch
for (inputs, target) in dataset.take(1):
    print("\n--- Forme d'un batch --- ")
    print(f"inputs['encoder_input'].shape: {inputs['encoder_input'].shape}")
    print(f"inputs['decoder_input'].shape: {inputs['decoder_input'].shape}")
    print(f"target.shape: {target.shape}")

## 6. Construction du Réseau de Neurones (Encodeur-Décodeur)

Nous construisons le modèle en utilisant l'API fonctionnelle de Keras. 

* **Couche d'Entrée (Anglais)**: `encoder_input`
* **Couche d'Embedding (Anglais)**: Transforme les entiers en vecteurs denses.
* **a. Encodeur (LSTM)**: Couche LSTM avec 256 états internes (`LATENT_DIM`). On récupère ses états `h` et `c`.
* **Couche d'Entrée (Français)**: `decoder_input`
* **Couche d'Embedding (Français)**: Une *autre* couche d'embedding pour le français.
* **b. Décodeur (LSTM)**: Utilise les états `h` et `c` de l'encodeur comme `initial_state`.
* **c. Couche Dense de Sortie**: Prédit le mot suivant. La taille est `VOCAB_SIZE` (2056) avec une activation `softmax`.

In [ ]:
# --- Définition de l'Encodeur ---
encoder_input = keras.Input(shape=(None,), name="encoder_input")

# Couche d'Embedding (remplace one-hot)
encoder_embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)
encoder_emb = encoder_embedding_layer(encoder_input)

# a. Couche LSTM de l'encodeur
encoder_lstm = LSTM(LATENT_DIM, return_state=True, name="encoder_lstm")
_, encoder_state_h, encoder_state_c = encoder_lstm(encoder_emb)

# Garder les états de l'encodeur
encoder_states = [encoder_state_h, encoder_state_c]

# --- Définition du Décodeur ---
decoder_input = keras.Input(shape=(None,), name="decoder_input")

# Couche d'Embedding (séparée pour le décodeur)
decoder_embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)
decoder_emb = decoder_embedding_layer(decoder_input)

# b. Couche LSTM du décodeur
# L'état initial est l'état de l'encodeur
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(decoder_emb, initial_state=encoder_states)

# c. Couche Dense de sortie
decoder_dense = Dense(VOCAB_SIZE, activation="softmax", name="decoder_dense")
decoder_output = decoder_dense(decoder_outputs)

# --- Création du Modèle (pour l'entraînement) ---
model = Model([encoder_input, decoder_input], decoder_output)

model.summary()

## 8. Compilation et Entraînement

Nous compilons le modèle. Notez l'utilisation de `sparse_categorical_crossentropy` : c'est la fonction de perte correcte lorsque vos cibles sont des entiers (ID de mots) et que votre sortie est une distribution de probabilité (softmax), ce qui évite d'avoir à convertir les cibles en one-hot.

In [ ]:
model.compile(
    optimizer="rmsprop", 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)

# L'entraînement peut prendre du temps (5-15 min sur un GPU Colab)
epochs = 20

history = model.fit(
    dataset,
    epochs=epochs,
    validation_data=dataset.take(int(0.1 * len(dataset))) # Utiliser une petite partie pour la validation
)

## 9. Création des Modèles d'Inférence

Pour la traduction (inférence), nous ne pouvons pas utiliser le *teacher forcing*. Nous devons prédire un mot à la fois et réinjecter cette prédiction dans le modèle pour générer le mot suivant.

Pour cela, nous divisons notre modèle entraîné en deux parties, comme vous l'avez décrit.

### 9.a. Modèle d'Encodeur

Prend la phrase en anglais et renvoie les états internes `(h, c)`.

In [ ]:
encoder_model = Model(encoder_input, encoder_states)
encoder_model.summary()

### 9.b. Modèle de Décodeur

Prend le mot précédent (ou `[start]`) et les états internes `(h, c)`, et renvoie la prédiction pour le mot suivant ainsi que les nouveaux états `(h, c)`.

In [ ]:
# Entrées pour les états du décodeur
decoder_state_input_h = keras.Input(shape=(LATENT_DIM,), name="decoder_state_h")
decoder_state_input_c = keras.Input(shape=(LATENT_DIM,), name="decoder_state_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Entrée pour le token (un seul mot à la fois)
decoder_input_single = keras.Input(shape=(1,), name="decoder_input_single")

# Réutiliser les couches entraînées
decoder_emb_inf = decoder_embedding_layer(decoder_input_single)
decoder_outputs_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm(
    decoder_emb_inf, initial_state=decoder_states_inputs
)
decoder_states_inf = [decoder_state_h_inf, decoder_state_c_inf]

decoder_output_inf = decoder_dense(decoder_outputs_inf)

# Création du modèle d'inférence
decoder_model = Model(
    [decoder_input_single] + decoder_states_inputs,
    [decoder_output_inf] + decoder_states_inf
)

decoder_model.summary()

### 9.c. Fonction de Traduction (Inférence)

C'est ici que nous assemblons le tout :
1.  Passer la séquence anglaise dans `encoder_model` pour obtenir les états.
2.  Commencer une boucle avec le token `[start]`.
3.  Utiliser `decoder_model` pour prédire le mot suivant.
4.  Récupérer l'ID du mot (avec `argmax`).
5.  Si c'est `[end]`, arrêter. Sinon, ajouter le mot à la traduction.
6.  Réinjecter le nouvel ID et les nouveaux états dans le `decoder_model` et continuer.

In [ ]:
# Créer des dictionnaires pour convertir les ID en mots (et vice-versa)
input_vocab = input_vectorization.get_vocabulary()
target_vocab = target_vectorization.get_vocabulary()

input_word_to_id = {word: i for i, word in enumerate(input_vocab)}
target_id_to_word = {i: word for i, word in enumerate(target_vocab)}

# Obtenir les ID des tokens spéciaux
start_token_id = target_vectorization(["[start]"]).numpy()[0, 0]
end_token_id = target_vectorization(["[end]"]).numpy()[0, 0]

def translate(input_sentence):
    # 1. Vectoriser la phrase d'entrée
    input_vec = input_vectorization([input_sentence])
    
    # 2. Obtenir les états initiaux de l'encodeur
    states_value = encoder_model.predict(input_vec, verbose=0)
    
    # 3. Préparer le premier token pour le décodeur ([start])
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = start_token_id

    decoded_sentence = ""
    stop_condition = False

    # Limite pour éviter les boucles infinies
    while not stop_condition:
        # 4. Prédire le mot suivant
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        
        # 5. Obtenir l'ID du mot prédit
        sampled_token_id = np.argmax(output_tokens[0, -1, :])
        
        # 6. Convertir l'ID en mot
        sampled_word = target_id_to_word.get(sampled_token_id, "[UNK]")
        
        # 7. Condition d'arrêt
        if sampled_word == "[end]" or len(decoded_sentence.split()) > max_target_seq_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + " "
            
        # 8. Mettre à jour la séquence cible (pour le prochain tour)
        target_seq[0, 0] = sampled_token_id
        
        # 9. Mettre à jour les états
        states_value = [h, c]
        
    return decoded_sentence.strip()

# --- Tester la traduction ---
print("\n===== Test de Traduction =====")
for i in [100, 200, 300, 400, 500]:
    input_text = input_texts[i]
    target_text = target_texts[i].replace("[start]", "").replace("[end]", "").strip()
    translation = translate(input_text)
    
    print("\n---------------------")
    print(f"Anglais : {input_text}")
    print(f"Français (vrai) : {target_text}")
    print(f"Français (prédit) : {translation}")